<a href="https://colab.research.google.com/github/Lucas-Buk/Machine-Learning-Metodos-Nao-Prob/blob/main/Trabalho_Ex_2_Regress%C3%A3o_Log%C3%ADstica_e_Algoritmo_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício 2 - Regressão Logística e Algoritmo KNN

Considere o dataset clássico sobre o diagnóstico de doenças cardíacas, disponível em:
https://www.kaggle.com/cherngs/heart-disease-cleveland-uci. O propósito básico deste
dataset é o de fornecer dados para treinar um classificador que preveja a ocorrência de
doença cardíaca em uma pessoa em função de 13 atributos (variáveis independentes).
Siga a tradição da comunidade mundial de Machine Learning e crie classificadores binários para o dataset usando regressão logística e o algoritmo KNN. Assim como no
exercício anterior, documente seu fluxo de trabalho, decisões e conclusões.

In [1]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
# Leitura dos dados externos
df2 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Pós/ML MNP/Datasets/heart_cleveland_upload.csv")
print(df2.shape)
df2.head()

(297, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [3]:
# Verificação do DataFrame
dados_vazios = df2.isnull().sum()
dados_vazios

age          0
sex          0
cp           0
trestbps     0
chol         0
fbs          0
restecg      0
thalach      0
exang        0
oldpeak      0
slope        0
ca           0
thal         0
condition    0
dtype: int64

Não há dados faltantes do DataFrame, assim o estudo pode ser realizado sem problemas.

In [4]:
# Seleção de dados
X = df2[['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']]   # Variáveis independentes

y = df2.condition  # Variável dependente

# Checagem das variáveis criadas
print(f'Dimensões X: {X.shape}')
print(f'Dimensões y: {y.shape}')

Dimensões X: (297, 13)
Dimensões y: (297,)


In [5]:
# Treinamento por regressão Logística
modeloRL = LogisticRegression(max_iter=1000)
modeloRL.fit(X,y)

# Exibição dos parâmetros do modelo
print(f'Beta0 = {modeloRL.intercept_}')
print(f'Beta1 = {modeloRL.coef_}')
print(f'Iterações: {modeloRL.n_iter_}')

Beta0 = [-4.19536356]
Beta1 = [[-0.01416251  1.04057186  0.55313154  0.02189226  0.00401221 -0.70385913
   0.2464007  -0.02163327  0.77772504  0.28575822  0.45905035  1.14537389
   0.72831994]]
Iterações: [710]


O número de iterações teve que ser alterado para a convergência completa do modelo.

In [6]:
# Avaliação da qualidade do modelo de regressão logística
# Calcula as previsões usando os dados de treino, sendo igual a todos os dados disponíveis
y_prevRL = modeloRL.predict(X)

# Matriz de confusão
print(f'Matriz de confusão:\n {confusion_matrix(y,y_prevRL)}')

# Relatório das métricas de qualidade
print(classification_report(y,y_prevRL,digits = 4))

Matriz de confusão:
 [[141  19]
 [ 26 111]]
              precision    recall  f1-score   support

           0     0.8443    0.8812    0.8624       160
           1     0.8538    0.8102    0.8315       137

    accuracy                         0.8485       297
   macro avg     0.8491    0.8457    0.8469       297
weighted avg     0.8487    0.8485    0.8481       297



O modelo de Regressão Logística criado apresentou um bom desempenho na classificação dos dados, com acurácia de aproximadamente 85%. O indíce de revocação apresentou valores mais distantes entres as classes 1 e 0, pois o modelo acertou muito mais os valores da classe 0 e, esses também estão em maior número, 160, contra 137 da classe 1.

In [7]:
# Capacidade de generalização do modelo de Regressão Logística
X_treino, X_teste, y_treino, y_teste = train_test_split(X,y,train_size = 0.8)

print(f'Dimensões dos dados de treino: {X_treino.shape}')
print(f'Dimensões dos dados de teste: {X_teste.shape}')

Dimensões dos dados de treino: (237, 13)
Dimensões dos dados de teste: (60, 13)


In [8]:
# Validação cruzada hold-out 
# Treina o modelo, apenas com os dados de treino
modeloRL_ho = LogisticRegression(max_iter=1000)
modeloRL_ho.fit(X_treino,y_treino)

y_prevRL_ho = modeloRL_ho.predict(X_teste)

# Matriz de confusão
print(f'Matriz de confusão:\n {confusion_matrix(y_teste,y_prevRL_ho)}')

# Relatório das métricas de qualidade
print(classification_report(y_teste,y_prevRL_ho,digits = 4))

Matriz de confusão:
 [[30  2]
 [ 4 24]]
              precision    recall  f1-score   support

           0     0.8824    0.9375    0.9091        32
           1     0.9231    0.8571    0.8889        28

    accuracy                         0.9000        60
   macro avg     0.9027    0.8973    0.8990        60
weighted avg     0.9014    0.9000    0.8997        60



O modelo de regressão logística apresenta uma boa generalização dos dados de teste, pois apresentou uma acurácia de aproximadamente 80% nos diversos testes realizados e no caso mostrado acima esse valor atingiu 87%.

In [9]:
# Classificação via algoritmo KNN
# Padronização da variável independente
esc = StandardScaler()
esc.fit(X)   
X_Norm = esc.transform(X)

# Treinamento do classificador KNN
modeloKNN = KNeighborsClassifier(n_neighbors = 3)
modeloKNN.fit(X_Norm,y)

# Avaliação da qualidade do classificador KNN (dados de treino = dados de teste)
y_prevKNN = modeloKNN.predict(X_Norm)

# Matriz de confusão
print(f'Matriz de confusão:\n {confusion_matrix(y,y_prevKNN)}')

# Relatório das métricas de qualidade
print(classification_report(y,y_prevKNN,digits = 4))

Matriz de confusão:
 [[145  15]
 [ 11 126]]
              precision    recall  f1-score   support

           0     0.9295    0.9062    0.9177       160
           1     0.8936    0.9197    0.9065       137

    accuracy                         0.9125       297
   macro avg     0.9116    0.9130    0.9121       297
weighted avg     0.9129    0.9125    0.9125       297



A classificação por KNN apresentou melhor desempenho em relação a por Regressão Logística, todos as métricas apresentaram uma melhor performance considereando a classificação com os 3 vizinhos mais próximos, aumentando esse valor a qualidade da classificação diminuiu. 

Nesse caso as métricas de qualidade apresentaram valores mais similares entre as classes, sendo a precisão a que apresenta a maior diferença, porque classificou mais elementos da classe 0, sem doença, como da classe 1.

In [10]:
# Avaliação da capacidade de generalização do classificador KNN
# Validação cruzada hold-out
X_treino, X_teste, y_treino, y_teste = train_test_split(X_Norm,y,train_size = 0.8)

print("Dimensões dos dados de treino:",X_treino.shape)
print("Dimensões dos dados de teste:",X_teste.shape)

Dimensões dos dados de treino: (237, 13)
Dimensões dos dados de teste: (60, 13)


In [11]:
# Treina o classificador com os dados de treino (80% do total)
modeloKNN_VC = KNeighborsClassifier(n_neighbors = 3)
modeloKNN_VC.fit(X_treino,y_treino)

# Avaliação da qualidade do classificador KNN
# Calcula as previsões usando os dados de teste (20% do total)
y_prevKNN_VC = modeloKNN_VC.predict(X_teste)

# Matriz de confusão
print(f'Matriz de confusão:\n {confusion_matrix(y_teste,y_prevKNN_VC)}')

# Relatório das métricas de qualidade
print(classification_report(y_teste,y_prevKNN_VC,digits = 4))

Matriz de confusão:
 [[27  4]
 [ 5 24]]
              precision    recall  f1-score   support

           0     0.8438    0.8710    0.8571        31
           1     0.8571    0.8276    0.8421        29

    accuracy                         0.8500        60
   macro avg     0.8504    0.8493    0.8496        60
weighted avg     0.8502    0.8500    0.8499        60



O modelo utilizando a validação cruzada hold-out apresentou uma boa capacidade de generalização do classificador KNN, com acurácia girando em torno de 80% nos testes realizados no modelo.

In [12]:
# Avaliação da capacidade de generalização do classificador KNN
# Validação cruzada k-folds

Aval = cross_val_score(modeloKNN,X_Norm,y,cv = 10,scoring = 'accuracy')
M = np.mean(Aval)            # Média das avaliações
DP = np.std(Aval,ddof = 1)   # Desvio padrão (amostral) das avaliações
CV_Aval = 100*DP/M           # Coeficiente de variação (%) das avaliações

print("Valores de acurácia obtidos:",Aval)
print(f'Média dos valores: {M}')
print(f'Coeficiente de variação: {CV_Aval:.4f}%')

Valores de acurácia obtidos: [0.63333333 0.83333333 0.9        0.93333333 0.76666667 0.86666667
 0.93333333 0.82758621 0.68965517 0.82758621]
Média dos valores: 0.8211494252873563
Coeficiente de variação: 12.1255%


Utilizando a validação cruzada k-folds o modelo teve uma acurácia média de 82% e uma variação de 12%, ou seja, baixa pois é < 20%. Com isso o modelo apresentou uma boa capacidade de generalização.

É interessante notar que dois valores obtidos para a acurácia foram abaixo de 70%, ainda assim obteve-se outros dois acima de 90% e a média maior que 82%.